# <font color='lightgreen'>Cyber Security Task 1 - EMBER Semi Supervised </font>
Shlomit Finegold,  Ilan Vasilevsky

---





# <font color='lightgreen'>Imports & functions</font>


In [ ]:
import pandas as pd # import pandas library
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def prep_data(origin_df):
  # split appeared to year and month
  origin_df[['year', 'month']] = origin_df['appeared'].str.split('-', 1, expand=True)
  new_df=origin_df.loc[origin_df['year']=='2018'].reset_index()
  new_df = new_df.drop(columns=['index'])
  # add categorial field = has avclass or not
  new_df['avclass_null']="has_value"
  new_df['avclass_null'].loc[new_df['avclass'].isnull()]="missing_value"
  #remove redundant
  new_df=new_df.drop(columns=['appeared','year'])
  return new_df

# <font color='lightgreen'>Load and prep data</font>


In [ ]:
#load train
path = "data/processed/train_v2.csv"
df_train = pd.read_csv(path)
df_train["origin"]="train"

In [ ]:
# load test
path = "data/processed/test.csv"
df_test = pd.read_csv(path)
df_test["origin"]="test"

In [ ]:
#concat train and test
df = pd.concat([df_train,df_test])

In [ ]:
#prep data
df_2018=prep_data(df)

# <font color='lightgreen'>Data description</font>

In [ ]:
print("data shape is: ",df_2018.shape)

In [ ]:
df_2018.sha256.nunique()

In [ ]:
df_2018 = df_2018.drop(columns=['index'])
df_2018.head()

In [ ]:
pd.set_option('display.max_columns', None)
df_2018.head()

## Data description -Visualisation 

### test-train split (per months)

In [ ]:
fig = px.pie(df_2018, names='origin',color_discrete_sequence=px.colors.qualitative.Pastel[4:6],title='origin dist',width=400, height=400)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

In [ ]:
month_grouped=df_2018.groupby(["month","origin"])["index"].count().reset_index(name="count")
fig = px.bar(month_grouped, x="month", y="count", color="origin", title="month split - train vs test",text_auto=True,color_discrete_sequence=px.colors.qualitative.Pastel[4:6],width=1000, height=400)
fig.show()

### label distrabution

In [ ]:
label_grouped=df_2018.groupby(["origin","label"])["index"].count().reset_index(name="count")
label_grouped['label']=label_grouped['label'].astype(str)
label_grouped['percent_label'] = 100 * label_grouped['count'] / label_grouped.groupby('origin')['count'].transform('sum')
fig = px.bar(label_grouped, x="origin", y="percent_label",title="label split per origin", color="label", text=label_grouped['percent_label'],color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(texttemplate='%{text:.4}%')
fig.update_layout(width=400,height=600)

fig.show()


### AV class distrabution

In [ ]:
avclass_grouped=df_2018.groupby(["avclass"])["index"].count().reset_index(name="count")
avclass_grouped['avclass'].loc[avclass_grouped['count']<3000]="low_count_av_class"
fig = px.bar(avclass_grouped.sort_values(by="count"), x="avclass", y="count", text_auto=True, title="avclass dist",color_discrete_sequence=px.colors.qualitative.Pastel[8:9])
fig.update_layout(width=1200,height=500)
fig.show()

In [ ]:
avclass_null_grouped=df_2018.groupby(["origin","avclass_null"])["index"].count().reset_index(name="count")
avclass_null_grouped['avclass_null']=avclass_null_grouped['avclass_null'].astype(str)
avclass_null_grouped['percent_avclass_null'] = 100 * avclass_null_grouped['count'] / avclass_null_grouped.groupby('origin')['count'].transform('sum')
fig = px.bar(avclass_null_grouped, x="origin", y="percent_avclass_null",title="has av class split per origin", color="avclass_null", text=avclass_null_grouped['percent_avclass_null'],color_discrete_sequence=px.colors.qualitative.Pastel[8:10])
fig.update_traces(texttemplate='%{text:.4}%')
fig.update_layout(width=400,height=600)
fig.show()

In [ ]:
avclass_null_label_grouped=df_2018.groupby(["avclass_null","label","origin"])["index"].count().reset_index(name="count")
avclass_null_label_grouped['label']=avclass_null_label_grouped['label'].astype(str)
avclass_null_label_grouped['percent_label'] = 100 * avclass_null_label_grouped['count'] / avclass_null_label_grouped.groupby(['avclass_null','origin'])['count'].transform('sum')

fig = px.bar(avclass_null_label_grouped, x="avclass_null", y="percent_label",title="label split per avclass-has value or not", color="label", text=avclass_null_label_grouped['percent_label'],color_discrete_sequence=px.colors.qualitative.Pastel, facet_col="origin")
fig.update_traces(texttemplate='%{text:.4}%')
fig.update_layout(width=1100,height=600,)

fig.show()

In [ ]:
df_2018.isna().sum()

# <font color='lightgreen'>Data exploration - for checking feature importance</font>

##Load train sampled data
 The exploration will be done on sampled data that will be used for the model as well.
 
 <font color='red'>**Ilan's to fill in how he sampled**</font>
Sampling 80k samples from train set: 
 1. for every row having an avclass but labeled as -1 we switched the label to 1
 2. rows having an empty string we put in unknown avclass
 3. we sampled 60k rows according to the distribution of avclasses having more than 10k rows. all other avlasses were categorized as 'other'. The last 20k samples were chosen randomly from 'other' avclass

In [ ]:
path = "data/processed/sample_train.csv"
sampled_train = pd.read_csv(path)

In [ ]:
clean_train=prep_data(sampled_train)

In [ ]:
print("data shape is: ",clean_train.shape)

In [ ]:
clean_train.head()

## av class

In [ ]:
avclass_grouped2=clean_train.groupby(["avclass"])["sha256"].count().reset_index(name="count")
fig = px.bar(avclass_grouped2.sort_values(by="count"), x="avclass", y="count", text_auto=True, title="avclass dist",color_discrete_sequence=px.colors.qualitative.Pastel[8:9])
fig.show()


In [ ]:
fig = px.pie(clean_train, names='avclass_null',color_discrete_sequence=px.colors.qualitative.Pastel[8:10],title='avclass_null dist',width=600, height=600)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

## Categorial features

In [ ]:
categiral_cols_lst=['has_debug','has_relocations','has_resources','has_signature','has_tls','month']

In [ ]:
for col in categiral_cols_lst:
  temp_grouped=clean_train.groupby([col,"label"])["sha256"].count().reset_index(name="count")
  temp_grouped['label']=temp_grouped['label'].astype(str)
  temp_grouped['percent_label'] = 100 * temp_grouped['count'] / temp_grouped.groupby(col)['count'].transform('sum')

  fig = px.bar(temp_grouped, x=col, y="percent_label",title="label split per "+col, color="label", text=temp_grouped['percent_label'],color_discrete_sequence=px.colors.qualitative.Pastel)
  fig.update_traces(texttemplate='%{text:.4}%')
  fig.update_layout(width=800,height=800,)

  fig.show()

## Categorial features

## Numeric features

In [ ]:
all_cols_lst=clean_train.columns.to_list()
index_n_labels_cols=['index','sha256','md5','label','avclass','avclass_null']
relevant_cols_lst = [col for col in all_cols_lst if col not in index_n_labels_cols]
numeric_cols_lst = [col for col in relevant_cols_lst if col not in categiral_cols_lst]

In [ ]:
for col in numeric_cols_lst:
  log_col=col+"_log"
  clean_train[log_col]=np.log10(clean_train[col]) # converting to log for better visualisation
  fig = px.violin(clean_train, y=log_col, x="label",box=True,color_discrete_sequence=px.colors.qualitative.Pastel, color="label",title="Box plot (violin) of log "+col)
  fig.update_layout(width=800,height=800,)
  fig.show()

## Feature selection and engineering

In [ ]:
clean_train['has_debug_relocations_resources_signature']=clean_train['has_debug']*clean_train['has_relocations']*clean_train['has_resources']*clean_train['has_signature']

In [ ]:
all_features_clean_train=clean_train

In [ ]:
clean_train=all_features_clean_train[['label','has_debug_relocations_resources_signature','size','vsize','symbols','numstrings','avlength','paths','urls','registry','MZ']]

### heatmap

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(clean_train)
scaled = scaler.transform(clean_train)
train_scaled = pd.DataFrame(scaled,columns=clean_train.columns.to_list())

In [ ]:

corr_matrix = train_scaled.corr()
fig = px.imshow(corr_matrix, text_auto=True)
fig.update_layout(width=1000,height=800,)
fig.show()

In [ ]:
clean_train=clean_train.drop(columns=['MZ','numstrings'])

### tsne

In [ ]:

features = clean_train[['has_debug_relocations_resources_signature','size','vsize','symbols','avlength','paths','urls','registry']]

tsne = TSNE(n_components=2, random_state=42)
projections = tsne.fit_transform(features)

fig = px.scatter(
    projections, x=0, y=1,
    color=clean_train.label, labels={'color': 'label'}
)
fig.show()

###<u>**Features selection explanation:**</u> 

<u> Categorial features:</u> 

- since we don't have all months and the trend is not super clear we can ignore this field

- since all the binary fields except has tls distrabute the same we can combine them into 1, has tls doesn't seem ot contribute a lot by itself

<u> Numeric features:</u> 
- printables seems to distrabute the same for lable 0 anfd 1 therefore not needed
- entropy seems to distrabute the same for lable 0 anfd 1 therefore not needed
- size is correlated to MZ and numstrings so we can remove MZ,numstirngs and keep only size (it's also a bit more correlated with the label than the other two)


In [ ]:
print("final data set for training:")
clean_train.head()

In [ ]:
# clean_train.to_csv('clean_sampled_train.csv')
# !cp clean_sampled_train.csv "/content/drive/MyDrive/Ai in Cybersecurity/task 1/data_files/clean_sampled_train.csv"